In [1]:
import os, requests, json, time
from tqdm import tqdm
# from multiprocessing import Pool
# from multiprocessing import Process, Queue
import multiprocessing as mp
from math import *

In [2]:
def jdump(obj): return json.dumps(obj, sort_keys=True, indent=4)
def jget(url, i):
    try:
        response = requests.get(url)
    except:
        quit()
    else:
        if response.status_code == 200:
            return json.loads(jdump(response.json()))
        elif i<5:
            print(str(response.status_code) + "; Failed to fetch data, trying again in one second.")
            time.sleep(1)
            jget(url, i+1)

In [4]:
f = open('../json/users.json')
users_info = json.load(f)
usernames = users_info['usernames']
follows = users_info['follows']
f.close()

In [5]:
userData = []
sampleData = []


def col(K):
    # print("\t\tProcess "+str(K))
    sample = 0
    username = usernames[K]
    valid = jget('https://api.scratch.mit.edu/accounts/checkusername/' +
                 username, 0)['msg'] == 'username exists'
    if valid:
        user = jget('https://api.scratch.mit.edu/users/'+username, 0)
        projects = 40
        off = 0
        while projects > 39:
            # print(username)
            userData.append(user)
            sample = jget('https://api.scratch.mit.edu/users/' +
                        username+'/projects?limit=40&offset=' + str(off), 0)
            # return sample
            # print("\t\tAppending "+username)
            sampleData.append(sample)
            # print(len(sample))
            projects = len(sample)
            off += 40

# count = len(follows)
count = 5

for K in tqdm(range(5)):
    col(K)

if False:
    size = mp.cpu_count()
    print(size)
    # pool = Pool(size)

    def main(Kl):
        pool = mp.Pool(size)
        print("\tmapping " + str(Kl) + " to col")
        result = pool.map(col, Kl)

    batches = ceil(len(follows)/size)
    for B in tqdm(range(batches)):
        M = B*size
        if __name__ == "__main__":
            main(range(M, M+size))
        print(str(B)+"/"+str(batches))

if False:
    for B in tqdm(range(ceil(len(follows)/size))):
        M = B*size
        if __name__ == "__main__":
            queue = Queue()

            processes = [Process(target=col, args=(K))
                         for K in range(M, M+size)]

            for p in processes:
                p.start()

            for p in processes:
                p.join()

if False:
    for B in tqdm(range(ceil(len(follows)/size))):
        M = B*size
        S = [pool.apply(col, args=(K,)) for K in range(M, M+size)]
        for s in S:
            sampleData.append(s)

if False:
    for K in tqdm(ceil(range(len(follows))/size)):
        # sample = col(K)
        # sample = pool.apply_async(col, (K,))
        M = K*size
        samples = pool.map(col, range(M, M+size-1))
        for sample in samples:
            sampleData.append(sample)

print("Data collected.")

100%|██████████| 5/5 [00:11<00:00,  2.33s/it]

Data collected.


In [6]:
data = 'User,,,,Average Project Statistics,,,,,,,\nUsername,Followers,Country,Join Date,Views,Loves,Favorites,Remixes,Public,Published,Visible,Commentable,Projects\n'

print("-- Processing data --")
for K in tqdm(range(len(userData))):
    # username = usernames[K]
    user = userData[K]
    username = user["username"]
    sample = sampleData[K]

    country = user['profile']['country']
    joinDate = user['history']['joined'].split('T')[0]

    views = 0
    loves = 0
    favorites = 0
    remixes = 0
    public = 0
    published = 0
    visible = 0
    commentable = 0
    count = 0

    for i in range(len(sample)):
        project = sample[i]
        views += project['stats']['views']
        loves += project['stats']['loves']
        favorites += project['stats']['favorites']
        remixes += project['stats']['remixes']
        public += 1 if project['public']else 0
        published += 1 if project['is_published']else 0
        visible += 1 if project['visibility'] == 'visible'else 0
        commentable += 1 if project['comments_allowed']else 0

        if project['public'] and project['is_published'] and project['visibility'] == 'visible':
            count += 1

    if count > 0:
        views = str(views/count)
        loves = str(loves/count)
        favorites = str(favorites/count)
        remixes = str(remixes/count)
        public = str(public/count)
        published = str(published/count)
        visible = str(visible/count)
        commentable = str(commentable/count)
        projects = str(count)

        data += username+','+follows[K].strip()+','+country+','+joinDate+','+views+','+loves + \
            ','+favorites+','+remixes+','+public+',' + \
            published+','+visible+','+commentable+","+projects+'\n'

f = open('dataset.csv', 'w')
f.write(data)
f.close()
print("File written to dataset.csv.")


-- Processing data --


100%|██████████| 9/9 [00:00<00:00, 5572.59it/s]

File written to dataset.csv.
